<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

# Задание 3

## Классификация текстов

В этом задании вам предстоит попробовать несколько методов, используемых в задаче классификации, а также понять насколько хорошо модель понимает смысл слов и какие слова в примере влияют на результат.

In [6]:
!pip install torchtext

     |████████████████████████████████| 8.0 MB 4.9 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.3 MB/s eta 0:00:37tcmalloc: large alloc 1147494400 bytes == 0x5621293ac000 @  0x7f92f9fe5615 0x5620ef7924cc 0x5620ef87247a 0x5620ef7952ed 0x5620ef886e1d 0x5620ef808e99 0x5620ef8039ee 0x5620ef796bda 0x5620ef808d00 0x5620ef8039ee 0x5620ef796bda 0x5620ef805737 0x5620ef887c66 0x5620ef804daf 0x5620ef887c66 0x5620ef804daf 0x5620ef887c66 0x5620ef804daf 0x5620ef797039 0x5620ef7da409 0x5620ef795c52 0x5620ef808c25 0x5620ef8039ee 0x5620ef796bda 0x5620ef805737 0x5620ef8039ee 0x5620ef796bda 0x5620ef804915 0x5620ef796afa 0x5620ef804c0d 0x5620ef8039ee
     |████████████████████████████████| 881.9 MB 20 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu111
    Uninstalling torch-1.9.0+cu111:
      Successfully uninstalled torch-1.9.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is 

In [33]:
import pandas as pd
import numpy as np
import torch

from torchtext.legacy import datasets

from torchtext.legacy.data import Field, LabelField
from torchtext.legacy.data import BucketIterator

from torchtext.vocab import Vectors, GloVe

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from tqdm.autonotebook import tqdm

from sklearn.metrics import f1_score

В этом задании мы будем использовать библиотеку torchtext. Она довольна проста в использовании и поможет нам сконцентрироваться на задаче, а не на написании Dataloader-а.

In [34]:
TEXT = Field(sequential=True, lower=True, include_lengths=True)  # Поле текста
LABEL = LabelField(dtype=torch.float)  # Поле метки

In [35]:
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Датасет на котором мы будем проводить эксперементы это комментарии к фильмам из сайта IMDB.

In [36]:
train, test = datasets.IMDB.splits(TEXT, LABEL)  # загрузим датасет
train, valid = train.split(random_state=random.seed(SEED))  # разобьем на части

In [37]:
TEXT.build_vocab(train)
LABEL.build_vocab(train)

In [38]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [39]:
train_iter, valid_iter, test_iter = BucketIterator.splits(
    (train, valid, test), 
    batch_size = 64,
    sort_within_batch = True,
    device = device)

## RNN

Для начала попробуем использовать рекурентные нейронные сети. На семинаре вы познакомились с GRU, вы можете также попробовать LSTM. Можно использовать для классификации как hidden_state, так и output последнего токена.

In [41]:
class RNNBaseline(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim, n_layers, dropout=dropout) #GRU(embedding_dim, hidden_dim)  # YOUR CODE GOES HERE
        
        self.fc = nn.Linear(hidden_dim, output_dim) # YOUR CODE GOES HERE
        
        #self.dropout_layer = nn.Dropout(p = dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
  
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        # cell arg for LSTM, remove for GRU
        packed_output, (hidden) = self.rnn(packed_embedded)#packed_output, (hidden, cell) = self.rnn(packed_embedded)
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)  

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        #print(hidden[-2,:,:].shape, hidden[-1,:,:].shape)
        hidden = hidden[-1,:,:]  # YOUR CODE GOES HERE
                
        #hidden = [batch size, hid dim * num directions] or [batch_size, hid dim * num directions]
            
        return self.fc(hidden)

Поиграйтесь с гиперпараметрами

In [42]:
vocab_size = len(TEXT.vocab)
emb_dim = 100
hidden_dim = 256
output_dim = 1
n_layers = 2
bidirectional = True
dropout = 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
patience=3

In [43]:
model = RNNBaseline(
    vocab_size=vocab_size,
    embedding_dim=emb_dim,
    hidden_dim=hidden_dim,
    output_dim=output_dim,
    n_layers=n_layers,
    bidirectional=bidirectional,
    dropout=dropout,
    pad_idx=PAD_IDX
)

In [44]:
model = model.to(device)

In [45]:
opt = torch.optim.Adam(model.parameters())
loss_func = nn.BCEWithLogitsLoss()

max_epochs = 20

Обучите сетку! Используйте любые вам удобные инструменты, Catalyst, PyTorch Lightning или свои велосипеды.

In [46]:
import numpy as np

min_loss = np.inf

cur_patience = 0

for epoch in range(1, max_epochs + 1):
    train_loss = 0.0
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar: 
      opt.zero_grad()
      input_embeds = batch.text[0].to(device)
      text_length = batch.text[1]
      labels = batch.label.to(device)
      prediction = model(input_embeds, text_length.to('cpu'))
      loss = loss_func(prediction.view(-1), labels)
      train_loss += loss
      loss.backward()
      opt.step()

    train_loss /= len(train_iter)
    val_loss = 0.0
    model.eval()
    pbar = tqdm(enumerate(valid_iter), total=len(valid_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar:
      input_embeds = batch.text[0].to(device)
      text_length = batch.text[1]
      labels = batch.label.to(device)
      prediction = model(input_embeds, text_length.to('cpu'))
      val_loss += loss_func(prediction.view(-1), labels)
    val_loss /= len(valid_iter)
    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))
model.load_state_dict(best_model)

  0%|          | 0/274 [00:00<?, ?it/s]

  0%|          | 0/118 [00:00<?, ?it/s]

Epoch: 1, Training Loss: 0.690378725528717, Validation Loss: 0.684716522693634


  0%|          | 0/274 [00:00<?, ?it/s]

  0%|          | 0/118 [00:00<?, ?it/s]

...

  0%|          | 0/118 [00:00<?, ?it/s]

Epoch: 7, Training Loss: 0.566494345664978, Validation Loss: 0.6258189678192139


  0%|          | 0/274 [00:00<?, ?it/s]

  0%|          | 0/118 [00:00<?, ?it/s]

<All keys matched successfully>

Посчитайте f1-score вашего классификатора на тестовом датасете.

**Ответ**:

In [47]:
labels = []
preds = []
with torch.no_grad():
  model.eval()
  for i, batch in enumerate(test_iter):
    input_embeds = batch.text[0].to(device)
    text_length = batch.text[1]
    label = batch.label.to(device)
    prediction = model(input_embeds, text_length.to('cpu'))
    qq = torch.tensor([1 if i > 0.5 else 0 for i in prediction.view(-1)])
    labels.append(label.detach().to('cpu').numpy())
    preds.append(qq.detach().to('cpu').numpy())
labels = np.concatenate(labels)
preds = np.concatenate(preds)
print(f1_score(labels, preds))

0.5983896030512785


## CNN

![](https://www.researchgate.net/publication/333752473/figure/fig1/AS:769346934673412@1560438011375/Standard-CNN-on-text-classification.png)

Для классификации текстов также часто используют сверточные нейронные сети. Идея в том, что как правило сентимент содержат словосочетания из двух-трех слов, например "очень хороший фильм" или "невероятная скука". Проходясь сверткой по этим словам мы получим какой-то большой скор и выхватим его с помощью MaxPool. Далее идет обычная полносвязная сетка. Важный момент: свертки применяются не последовательно, а параллельно. Давайте попробуем!

In [49]:
TEXT = Field(sequential=True, lower=True, batch_first=True)  # batch_first тк мы используем conv  
LABEL = LabelField(batch_first=True, dtype=torch.float)

train, tst = datasets.IMDB.splits(TEXT, LABEL)
trn, vld = train.split(random_state=random.seed(SEED))

TEXT.build_vocab(trn)
LABEL.build_vocab(trn)

device = "cuda" if torch.cuda.is_available() else "cpu"


In [50]:
train_iter, val_iter, test_iter = BucketIterator.splits(
        (trn, vld, tst),
        batch_sizes=(128, 256, 256),
        sort=False,
        sort_key= lambda x: len(x.src),
        sort_within_batch=False,
        device=device,
        repeat=False,
)

Вы можете использовать Conv2d с `in_channels=1, kernel_size=(kernel_sizes[0], emb_dim))` или Conv1d c `in_channels=emb_dim, kernel_size=kernel_size[0]`. Но хорошенько подумайте над shape в обоих случаях.

In [51]:
class CNN(nn.Module):
    def __init__(
        self,
        vocab_size,
        emb_dim,
        out_channels,
        kernel_sizes,
        dropout=0.5,
    ):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, emb_dim)
        self.conv_0 = nn.Conv2d(in_channels=1, out_channels = out_channels, kernel_size=(kernel_sizes[0], emb_dim))  # YOUR CODE GOES HERE
        
        self.conv_1 = nn.Conv2d(in_channels=1, out_channels = out_channels, kernel_size=(kernel_sizes[1], emb_dim))  # YOUR CODE GOES HERE
        
        self.conv_2 = nn.Conv2d(in_channels=1, out_channels = out_channels, kernel_size=(kernel_sizes[2], emb_dim))  # YOUR CODE GOES HERE
        
        self.fc = nn.Linear(len(kernel_sizes) * out_channels, 1)
        
        self.dropout = nn.Dropout(p = dropout)
        
        
    def forward(self, text):
        
        embedded = self.embedding(text)
        embedded = embedded.view( embedded.shape[0], 1, embedded.shape[1], 300)#(5, 1, 3, 300, -1))  # may be reshape here
        
        conved_0 = F.relu(self.conv_0(embedded))[:, :, :, 0]  # may be reshape here
        conved_1 = F.relu(self.conv_1(embedded))[:, :, :, 0]  # may be reshape here
        conved_2 = F.relu(self.conv_2(embedded))[:, :, :, 0]  # may be reshape here

        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        
        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim=1))
            
        return self.fc(cat)

In [52]:
kernel_sizes = [3, 4, 5]
vocab_size = len(TEXT.vocab)
out_channels=16
dropout = 0.5
dim = 300

model = CNN(vocab_size=vocab_size, emb_dim=dim, out_channels=out_channels,
            kernel_sizes=kernel_sizes, dropout=dropout)

In [53]:
model.to(device)

CNN(
  (embedding): Embedding(201614, 300)
  (conv_0): Conv2d(1, 16, kernel_size=(3, 300), stride=(1, 1))
  (conv_1): Conv2d(1, 16, kernel_size=(4, 300), stride=(1, 1))
  (conv_2): Conv2d(1, 16, kernel_size=(5, 300), stride=(1, 1))
  (fc): Linear(in_features=48, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [54]:
opt = torch.optim.Adam(model.parameters())
loss_func = nn.BCEWithLogitsLoss()

In [55]:
max_epochs = 10

Обучите!

In [56]:
import numpy as np

min_loss = np.inf

cur_patience = 0

for epoch in range(1, max_epochs + 1):
    train_loss = 0.0
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar: 
      opt.zero_grad()
      input_embeds = batch.text
      prediction = model(input_embeds)
      labels = batch.label
      loss = loss_func(prediction.view(-1), labels)
      train_loss += loss
      loss.backward()
      opt.step()
      labels.to('cpu')
      input_embeds.to('cpu')
      torch.cuda.empty_cache()
    train_loss /= len(train_iter)
    val_loss = 0.0
    model.eval()
    pbar = tqdm(enumerate(val_iter), total=len(val_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar:
      with torch.no_grad():
        input_embeds = batch.text
        prediction = model(input_embeds)
        labels = batch.label
        val_loss += loss_func(prediction.view(-1), labels)
        labels.to('cpu')
        input_embeds.to('cpu')
        torch.cuda.empty_cache()
    val_loss /= len(val_iter)
    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))
model.load_state_dict(best_model)

  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1, Training Loss: 0.6795055866241455, Validation Loss: 0.5548259019851685


  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

...

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 8, Training Loss: 0.23573091626167297, Validation Loss: 0.4930281341075897


  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

<All keys matched successfully>

In [57]:
labels = []
preds = []
with torch.no_grad():
  model.eval()
  for i, batch in enumerate(test_iter):
    input_embeds = batch.text.to(device)
    label = batch.label.to(device)
    prediction = model(input_embeds)
    qq = torch.tensor([1 if x > 0.5 else 0 for x in prediction.view(-1)]) 
    labels.append(label.detach().to('cpu').numpy())
    preds.append(qq.detach().to('cpu').numpy())
labels = np.concatenate(labels)
preds = np.concatenate(preds)
print(f1_score(labels, preds))

0.8020565552699229


Посчитайте f1-score вашего классификатора.

**Ответ**: 

## Интерпретируемость

Посмотрим, куда смотрит наша модель. Достаточно запустить код ниже.

In [58]:
!pip install -q captum

In [59]:
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

PAD_IND = TEXT.vocab.stoi['pad']

token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(model, model.embedding)

In [60]:
def forward_with_softmax(inp):
    logits = model(inp)
    return torch.softmax(logits, 0)[0][1]

def forward_with_sigmoid(input):
    return torch.sigmoid(model(input))


# accumalate couple samples in this array for visualization purposes
vis_data_records_ig = []

def interpret_sentence(model, sentence, min_len = 7, label = 0):
    model.eval()
    text = [tok for tok in TEXT.tokenize(sentence)]
    if len(text) < min_len:
        text += ['pad'] * (min_len - len(text))
    indexed = [TEXT.vocab.stoi[t] for t in text]

    model.zero_grad()

    input_indices = torch.tensor(indexed, device=device)
    input_indices = input_indices.unsqueeze(0)
    
    # input_indices dim: [sequence_length]
    seq_length = min_len

    # predict
    pred = forward_with_sigmoid(input_indices).item()
    pred_ind = round(pred)

    # generate reference indices for each sample
    reference_indices = token_reference.generate_reference(seq_length, device=device).unsqueeze(0)

    # compute attributions and approximation delta using layer integrated gradients
    attributions_ig, delta = lig.attribute(input_indices, reference_indices, \
                                           n_steps=5000, return_convergence_delta=True)

    print('pred: ', LABEL.vocab.itos[pred_ind], '(', '%.2f'%pred, ')', ', delta: ', abs(delta))

    add_attributions_to_visualizer(attributions_ig, text, pred, pred_ind, label, delta, vis_data_records_ig)
    
def add_attributions_to_visualizer(attributions, text, pred, pred_ind, label, delta, vis_data_records):
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    # storing couple samples in an array for visualization purposes
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            LABEL.vocab.itos[pred_ind],
                            LABEL.vocab.itos[label],
                            LABEL.vocab.itos[1],
                            attributions.sum(),       
                            text,
                            delta))

In [61]:
interpret_sentence(model, 'It was a fantastic performance !', label=1)
interpret_sentence(model, 'Best film ever', label=1)
interpret_sentence(model, 'Such a great show!', label=1)
interpret_sentence(model, 'It was a horrible movie', label=0)
interpret_sentence(model, 'I\'ve never watched something as bad', label=0)
interpret_sentence(model, 'It is a disgusting movie!', label=0)

pred:  pos ( 0.96 ) , delta:  tensor([0.0002], device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.57 ) , delta:  tensor([7.0738e-06], device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.84 ) , delta:  tensor([0.0001], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.31 ) , delta:  tensor([3.0094e-06], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.11 ) , delta:  tensor([0.0001], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.11 ) , delta:  tensor([7.9793e-05], device='cuda:0', dtype=torch.float64)


Попробуйте добавить свои примеры!

In [62]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.96),pos,1.51,It was a fantastic performance ! pad
pos,pos (0.57),pos,0.80,Best film ever pad pad pad pad
pos,pos (0.84),pos,1.07,Such a great show! pad pad pad
neg,neg (0.31),pos,0.05,It was a horrible movie pad pad
neg,neg (0.11),pos,-0.41,I've never watched something as bad pad
neg,neg (0.11),pos,-0.51,It is a disgusting movie! pad pad


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.96),pos,1.51,It was a fantastic performance ! pad
pos,pos (0.57),pos,0.80,Best film ever pad pad pad pad
pos,pos (0.84),pos,1.07,Such a great show! pad pad pad
neg,neg (0.31),pos,0.05,It was a horrible movie pad pad
neg,neg (0.11),pos,-0.41,I've never watched something as bad pad
neg,neg (0.11),pos,-0.51,It is a disgusting movie! pad pad


## Эмбеддинги слов

Вы ведь не забыли, как мы можем применить знания о word2vec и GloVe. Давайте попробуем!

In [63]:
TEXT.build_vocab(trn, vectors=GloVe(name='6B', dim=300))# YOUR CODE GOES HERE
# подсказка: один из импортов пока не использовался, быть может он нужен в строке выше :)
LABEL.build_vocab(trn)

word_embeddings = TEXT.vocab.vectors

kernel_sizes = [3, 4, 5]
vocab_size = len(TEXT.vocab)
dropout = 0.5
dim = 300

In [64]:
train, tst = datasets.IMDB.splits(TEXT, LABEL)
trn, vld = train.split(random_state=random.seed(SEED))



train_iter, val_iter, test_iter = BucketIterator.splits(
        (trn, vld, tst),
        batch_sizes=(128, 256, 256),
        sort=False,
        sort_key= lambda x: len(x.src),
        sort_within_batch=False,
        device=device,
        repeat=False,
)

In [65]:
model = CNN(vocab_size=vocab_size, emb_dim=dim, out_channels=16,
            kernel_sizes=kernel_sizes, dropout=dropout)

word_embeddings = TEXT.vocab.vectors

prev_shape = model.embedding.weight.shape

model.embedding.weight = nn.parameter.Parameter(word_embeddings) # инициализируйте эмбэдинги

assert prev_shape == model.embedding.weight.shape
model.to(device)

opt = torch.optim.Adam(model.parameters())

Вы знаете, что делать.

In [66]:
import numpy as np

min_loss = np.inf

cur_patience = 0

for epoch in range(1, max_epochs + 1):
    train_loss = 0.0
    model.train()
    pbar = tqdm(enumerate(train_iter), total=len(train_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar: 
      opt.zero_grad()
      input_embeds = batch.text
      prediction = model(input_embeds)
      labels = batch.label
      loss = loss_func(prediction.view(-1), labels)
      train_loss += loss
      loss.backward()
      opt.step()
      torch.cuda.empty_cache()
    train_loss /= len(train_iter)
    val_loss = 0.0
    model.eval()
    pbar = tqdm(enumerate(val_iter), total=len(val_iter), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    for it, batch in pbar:
      with torch.no_grad():
        input_embeds = batch.text
        prediction = model(input_embeds)
        labels = batch.label
        val_loss += loss_func(prediction.view(-1), labels)
        torch.cuda.empty_cache()
    val_loss /= len(val_iter)
    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))
model.load_state_dict(best_model)

  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1, Training Loss: 0.5627465844154358, Validation Loss: 0.43090569972991943


  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

...

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 6, Training Loss: 0.061750900000333786, Validation Loss: 0.3705313503742218


  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

<All keys matched successfully>

Посчитайте f1-score вашего классификатора.

**Ответ**:

In [67]:

labels = []
preds = []
with torch.no_grad():
  model.eval()
  for i, batch in enumerate(test_iter):
    input_embeds = batch.text.to(device)
    label = batch.label.to(device)
    prediction = model(input_embeds)
    qq = torch.tensor([1 if x > 0.5 else 0 for x in prediction.view(-1)]) 
    labels.append(label.detach().to('cpu').numpy())
    preds.append(qq.detach().to('cpu').numpy())
labels = np.concatenate(labels)
preds = np.concatenate(preds)
print(f1_score(labels, preds))

0.8360150062526053


Проверим насколько все хорошо!

In [68]:
PAD_IND = TEXT.vocab.stoi['pad']

token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(model, model.embedding)
vis_data_records_ig = []

interpret_sentence(model, 'It was a fantastic performance !', label=1)
interpret_sentence(model, 'Best film ever', label=1)
interpret_sentence(model, 'Such a great show!', label=1)
interpret_sentence(model, 'It was a horrible movie', label=0)
interpret_sentence(model, 'I\'ve never watched something as bad', label=0)
interpret_sentence(model, 'It is a disgusting movie!', label=0)

pred:  pos ( 0.98 ) , delta:  tensor([0.0003], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.12 ) , delta:  tensor([3.1395e-05], device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.78 ) , delta:  tensor([0.0003], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.00 ) , delta:  tensor([1.6622e-06], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.44 ) , delta:  tensor([0.0003], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.01 ) , delta:  tensor([2.6157e-05], device='cuda:0', dtype=torch.float64)


In [69]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.98),pos,1.82,It was a fantastic performance ! pad
pos,neg (0.12),pos,1.21,Best film ever pad pad pad pad
pos,pos (0.78),pos,1.60,Such a great show! pad pad pad
neg,neg (0.00),pos,-0.43,It was a horrible movie pad pad
neg,neg (0.44),pos,1.20,I've never watched something as bad pad
neg,neg (0.01),pos,-0.36,It is a disgusting movie! pad pad


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.98),pos,1.82,It was a fantastic performance ! pad
pos,neg (0.12),pos,1.21,Best film ever pad pad pad pad
pos,pos (0.78),pos,1.60,Such a great show! pad pad pad
neg,neg (0.00),pos,-0.43,It was a horrible movie pad pad
neg,neg (0.44),pos,1.20,I've never watched something as bad pad
neg,neg (0.01),pos,-0.36,It is a disgusting movie! pad pad
